**NOTE:** You might need to run this notebook locally for an optimal experience, since it contains javascript code that cannot be run on colab/github/etc.

# Query a pre-built database of chess positions

This demo shows you how to search a pre-bulit database of chess positions for positions that are similar to your own query position.

### Enter your query

First you need to specify the position for which you want to find similar positions for. Use the widget in the following cell to set up a board and then press the *Export Position* button.

In [ ]:
'''RUN THIS CELL ONLY ONCE to create the widget and a query queue'''
query = []
from IPython.display import HTML
HTML('input-widget.html')

Run the next cell to add your position to the query queue.

In [ ]:
'''ALWAYS RUN THIS CELL AFTER YOU ENTER A POSITION IN THE PREVIOUS WIDGET'''
print(f"The fen string of your position is: {fen}.")
query.append(fen)
print(f"The fen string was added to the query, which now has {len(query)} positions stored.")

After executing the last two cells you can **view the current query queue with the command below**. You can also **add more positions to the query queue by using the widget again and then executing the previous cell again**.

In [ ]:
print(query)

**Alternatively** if the widget above does not work you can visit [lichess.org](https://lichess.org/editor) retrieve the fen string of your position there and then paste it in the array below.

In [ ]:
'''
EXECUTE THIS CELL ONLY IF YOU DON'T WANT TO USE THE WIDGET ABOVE.
Replace the example fen string with your own.
'''
query = [
    "rnbq1rk1/pp2bppp/2p1pn2/4N1B1/2pP4/2N3P1/PP2PPBP/R2Q1RK1 b Qq - 0 1", #list of fen strings seperated by comma
    "rnbq1rk1/pp2bppp/2p1pn2/4N1B1/2pP4/2N3P1/PP2PPBP/R2Q1RK1 b Qq - 0 1"
]

### Load the database of chess positions

In [ ]:
from chesspos.binary_index import index_load, index_query_positions

Let's download a small precompiled database from google drive and uncompress it.

In [ ]:
!curl -L -o '../data/index_2013.faiss.bz2' 'https://docs.google.com/uc?export=download&id=1MQKJ6KSmYRyPbIP1ldsNBo-0dGhi-CpQ'
!bzip2 -d ../data/index_2013.faiss.bz2

Now load the database into memory. *You need at least 256MB of RAM.*

In [ ]:
filepath = "../data/index_2013.faiss"
index = index_load(filepath, is_binary=True)
print(f"The database you loaded contains {round(index.ntotal/1.e6,3)} million positions")

Specify the **expected number of results per query**, then search the database and retrieve most similar positions.

In [ ]:
search_results = 10
dist, reconstructed = index_query_positions(query, index, input_format='fen', output_format='fen',
                                            num_results=search_results)

### Inspect the retrieved results

Execute the cell below. This will generate a widget which lets you inspect the retireved queries.

In [ ]:
from IPython.display import HTML
html = '''<link rel="stylesheet" href="https://unpkg.com/@chrisoakman/chessboardjs@1.0.0/dist/chessboard-1.0.0.min.css" integrity="sha384-q94+BZtLrkL1/ohfjR8c6L+A6qzNH9R2hBLwyoAfu3i/WCvQjzL2RQJ3uNHDISdU" crossorigin="anonymous"><table>'''
for i in range(len(reconstructed)):
    html += f'''<tr><td>Your Query Position {i}</td><td><span>The (Hamming) distance between query and </span><select id="mySelect{i}" onchange="myFunction{i}()">'''
    for j in range(search_results):
        html += f'''<option value='{reconstructed[i][j]}|{dist[i][j]}'>Similar Position {j}</option>'''
    html += f'''</select><span> is </span><span id="dist{i}">0</span><span>.</span></td></tr><tr><td><div id="query{i}" style="width: 400px"></div></td><td><div id="myBoard{i}" style="width: 400px"></div></td></tr>'''
html += '''</table><script src="https://unpkg.com/@chrisoakman/chessboardjs@1.0.0/dist/chessboard-1.0.0.min.js" integrity="sha384-8Vi8VHwn3vjQ9eUHUxex3JSN/NFqUg3QbPyX8kWyb93+8AC/pPWTzj+nHtbC5bxD" crossorigin="anonymous"></script><script>'''
for i in range(len(reconstructed)):
    html += f'''var pos{i} = document.getElementById("mySelect{i}").value;var board{i} = Chessboard('myBoard{i}',{{showNotation: false}});var query{i} = Chessboard('query{i}',{{position: '{query[i]}',showNotation: false}});function myFunction{i}() {{var infos = document.getElementById("mySelect{i}").value;var position = infos.split("|")[0];var distance = infos.split("|")[1];board{i}.position(position);document.getElementById("dist{i}").innerHTML = distance;}}'''
html += '''</script>'''
HTML(html)

## How are similar positions calculated?

Positions are internally represented as bitboards, where each combination of (sqare,piece) is assigned the value 'true' if a piece of this type occupies the square and 'false' otherwise.

Therefore the distance between positions is the [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance) between boolean vectors.

This measure of position distances is however not too useful as you can find out by experimenting with the above demo. For other ways of measuring position similarity check out the metric learning part of this repo.